In [ ]:
!pip install fake_useragent

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
from fake_useragent import UserAgent
from tqdm.auto import tqdm
import pandas as pd
import numpy as np

In [ ]:
ua = UserAgent()
session = requests.session()

In [ ]:
def parse_one_article(block):

    # создаем запрос для отдельной страницы с рецептом

    req = session.get(block['ссылка на статью'],
                      headers={'User-Agent': ua.random})
    page = req.text

    # парсим страницу
    try:

        soup = BeautifulSoup(page, 'html.parser')
        text = soup.get_text()

    # проверяем наличие слова "калорийность" в тексте
        if 'Калорийность' in text:

            block['калорийность (ккал)'] = int(soup.find('span',
                                                        {'itemprop':
                                                        'calories'}
                                                        ).text.replace(' ккал', ''))
            block['белки (г)'] = int(soup.find('span',
                                            {'itemprop':
                                                'proteinContent'}
                                            ).text.replace(' г', ''))
            block['жиры (г)'] = int(soup.find('span',
                                            {'itemprop':
                                            'fatContent'}).text.replace(' г', ''))
            block['углеводы (г)'] = int(soup.find('span',
                                                {'itemprop':
                                                'carbohydrateContent'}
                                                ).text.replace(' г', ''))
            block['ингредиенты'] = []
            ingredients = soup.find_all('li',{'class': 'ingredient flex-dot-line'})

            for ingredient in ingredients:
                name = ingredient.find('span', {'class': 'name'}).text.strip()

                block['ингредиенты'].append(name)


    # поскольку текст рецепта разбит по блокам, то перед добавлением
    # в датафрейм необходимо собрать его части в одну строку, при этом почистив
    # части от личшних пробелов и переносов строк.

            block['рецепт'] = ' '.join(
                [i.text.strip() for i in
                soup.find_all('div',{'class': 'detailed_step_description_big'})])
    except Exception:
        pass

    return block


In [ ]:
def get_nth_page(page_number):

    # создаем запрос для страницы с рецептами

    url = f'https://povar.ru/mostnew/all/{page_number}/'
    req = session.get(url, headers={'User-Agent': ua.random})
    page = req.text

    # парсим страницу

    soup = BeautifulSoup(page, 'html.parser')
    news = soup.find_all('div', {'class': 'recipe'})

    # создаем список blocks, где будут храниться все собранные данные

    blocks = []


    for n in news:
        try:
            blocks.append(parse_news_page_block(n))
        except Exception as e:
                pass

    return blocks

In [ ]:
def parse_news_page_block(one_block):

    # со страницы с рецептами собираются данные
    # для каждого поста и добавляются в словарь block

    block = {}

    title_obj = one_block.find('a')
    block['название статьи'] = title_obj.text
    block['ссылка на статью'] = f"https://povar.ru{title_obj.attrs['href']}"

    return parse_one_article(block)


In [ ]:
def run_all(start, finish):
    blocks = []
    for i in tqdm(range(start, finish)):
        blocks.extend(get_nth_page(i))

    return blocks

In [ ]:
# из-за того, что на сайте было очень много страниц, пришлось разделить
# процесс сбора данных на несколько этапов по 300 страниц
blocks_0 = run_all(1, 300)

  0%|          | 0/299 [00:00<?, ?it/s]

In [ ]:
df = pd.DataFrame(blocks_0)
df.to_csv('/content/my_recipes.csv', index=True)

In [ ]:
blocks_1 = run_all(300, 600)

  0%|          | 0/300 [00:00<?, ?it/s]

In [ ]:
df = pd.read_csv("my_recipes.csv")
df_1 = pd.DataFrame(blocks_1)
df = pd.concat([df, df_1])
df.to_csv('/content/my_recipes.csv', index=True)

In [ ]:
blocks_2 = run_all(600, 900)

  0%|          | 0/300 [00:00<?, ?it/s]

In [ ]:
df = pd.read_csv("my_recipes.csv")

In [ ]:
df_2 = pd.DataFrame(blocks_2)
df = pd.concat([df, df_2])
df.to_csv('/content/my_recipes.csv', index=True)

In [ ]:
blocks_2 = run_all(900, 1200)

  0%|          | 0/300 [00:00<?, ?it/s]

In [ ]:
df = pd.read_csv("my_recipes.csv")

In [ ]:
df_2 = pd.DataFrame(blocks_2)
df = pd.concat([df, df_2])
df.to_csv('/content/my_recipes.csv', index=True)

In [ ]:
blocks_3 = run_all(1200, 1500)

  0%|          | 0/300 [00:00<?, ?it/s]

In [ ]:
df = pd.read_csv("my_recipes.csv")

In [ ]:
df_3 = pd.DataFrame(blocks_3)
df = pd.concat([df, df_3])
df.to_csv('/content/my_recipes.csv', index=True)

In [ ]:
blocks_4 = run_all(1500, 1800)

  0%|          | 0/300 [00:00<?, ?it/s]

In [ ]:
df = pd.read_csv("my_recipes.csv")

In [ ]:
df_4 = pd.DataFrame(blocks_4)
df = pd.concat([df, df_4])
df.to_csv('/content/my_recipes.csv')

In [ ]:
blocks_5 = run_all(1800, 2100)

  0%|          | 0/300 [00:00<?, ?it/s]

In [ ]:
df = pd.read_csv("my_recipes.csv")

In [ ]:
df_5 = pd.DataFrame(blocks_5)
df = pd.concat([df, df_5])
df.to_csv('/content/my_recipes.csv')

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("my_recipes.csv")

In [ ]:
df

,Unnamed: 0.6,Unnamed: 0.5,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,название статьи,ссылка на статью,калорийность (ккал),белки (г),жиры (г),углеводы (г),ингредиенты,рецепт
0,0,0.0,0.0,0.0,0.0,0.0,0.0,"Салат ""Марианна""",https://povar.ru/recipes/salat_marianna-93701....,239.0,16.0,14.0,10.0,"['Ветчина', 'Сыр твердый', 'Огурец', 'Черносли...",Подготовьте ингредиенты. Свежий огурец ополосн...
1,1,1.0,1.0,1.0,1.0,1.0,1.0,Суп с килькой в томате с вермишелью,https://povar.ru/recipes/sup_s_kilkoi_v_tomate...,144.0,4.0,9.0,11.0,"['Килька', 'Вермишель', 'Лук', 'Морковь', 'Кар...",Подготовьте все ингредиенты. Макаронные издели...
2,2,2.0,2.0,2.0,2.0,2.0,2.0,"Салат ""Астория""",https://povar.ru/recipes/salat_astoriya-93713....,116.0,5.0,9.0,5.0,"['Ветчина', 'Шампиньоны', 'Лук', 'Картофель', ...",Подготовьте ингредиенты. Очистите лук и нарежь...
3,3,3.0,3.0,3.0,3.0,3.0,3.0,"Спагетти ""Четыре помидора""",https://povar.ru/recipes/spagetti_chetyre_pomi...,144.0,2.0,7.0,16.0,[],NaN
4,4,4.0,4.0,4.0,4.0,4.0,4.0,Пельмени в сметане в мультиварке,https://povar.ru/recipes/pelmeni_v_smetane_v_m...,266.0,6.0,18.0,16.0,"['Пельмени', 'Сметана', 'Вода', 'Растительное ...",Подготовьте все ингредиенты. Можете взять гото...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83955,11995,NaN,NaN,NaN,NaN,NaN,NaN,Мексиканские кексы,https://povar.ru/recipes/meksikanskie_keksy-39...,NaN,NaN,NaN,NaN,NaN,NaN
83956,11996,NaN,NaN,NaN,NaN,NaN,NaN,Форель Piemonte,https://povar.ru/recipes/forel_Piemonte-305.html,NaN,NaN,NaN,NaN,NaN,NaN
83957,11997,NaN,NaN,NaN,NaN,NaN,NaN,Булочки Cantucci,https://povar.ru/recipes/bulochki_Cantucci-297...,406.0,14.0,15.0,50.0,"['Масло', 'Сахар', 'Яйца', 'Соль', 'Ваниль', '...",Смешайте масло и сахарную пудру. Добавьте яйца...
83958,11998,NaN,NaN,NaN,NaN,NaN,NaN,Кусочки оленины с апельсиновым ароматом,https://povar.ru/recipes/kusochki_oleniny_s_ap...,93.0,6.0,5.0,3.0,"['Оленина', 'Помидоры', 'Грибы', 'Сало', 'Сухо...",Разрежьте на куски оленину и поместите их в ми...


In [ ]:
# чистим датафрейм от ненужных данных
df.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.2',
         'Unnamed: 0.3','Unnamed: 0.4','Unnamed: 0.5',
         'Unnamed: 0.6'], axis= 1 , inplace= True )

In [ ]:
df = df.dropna()

In [ ]:
df.to_csv('/content/my_recipes.csv')